In [1]:
# Cell 1: Import Libraries and Load Tokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_scheduler
from datasets import load_dataset
import pandas as pd
import wandb
from torch.cuda.amp import autocast, GradScaler
import gc
from tqdm import tqdm

# Initialize WandB in offline mode
wandb.init(project="DeBERTa-Fake-News", config={"epochs": 3, "batch_size": 32, "lr": 1e-5}, mode="offline")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Free GPU cache
torch.cuda.empty_cache()
gc.collect()

# Load DeBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
# Cell 2: Load Datasets
# Load dataset from Hugging Face for pre-training
ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1", split="train")
pretrain_texts = ds["text"][:int(0.05 * len(ds["text"]))]  # Use 10% of Wikitext-103
pretrain_df = pd.DataFrame({"text": pretrain_texts, "label": [0] * len(pretrain_texts)})

# Load dataset from CSV (WELFake) for fine-tuning
welfake_df = pd.read_csv("/kaggle/input/welfake-dataset-for-fake-news/WELFake_Dataset.csv", usecols=["text", "label"], dtype={"label": str})

# Ensure labels are numeric & clean
welfake_df["label"] = welfake_df["label"].str.replace(r"[^\d]", "", regex=True).str.strip()
welfake_df = welfake_df[welfake_df["label"] != ""]  # Remove empty values
welfake_df["label"] = welfake_df["label"].astype(int)  # Convert to int for PyTorch
print("Data loading complete!")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Data loading complete!


In [3]:
# Cell 3: Define Dataset Classes
class MLMDataset(Dataset):
    def __init__(self, texts, max_len=128):
        self.encodings = tokenizer(texts, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
        self.labels = self.encodings.input_ids.clone()
        rand = torch.rand(self.labels.shape)
        mask_arr = (rand < 0.15) * (self.labels != tokenizer.pad_token_id) * (self.labels != tokenizer.cls_token_id) * (self.labels != tokenizer.sep_token_id)
        self.encodings.input_ids[mask_arr] = tokenizer.mask_token_id
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}, self.labels[idx]

class FakeNewsDataset(Dataset):
    def __init__(self, df, max_len=256):
        self.labels = torch.tensor(df["label"].values, dtype=torch.long)
        self.encodings = tokenizer(df["text"].astype(str).tolist(), padding="max_length", truncation=True, 
                                   max_length=max_len, return_tensors="pt")
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items() if key != "token_type_ids"}, self.labels[idx]

In [4]:
# Cell 4: Define DeBERTa Model
class DeBERTaClassifier(nn.Module):
    def __init__(self, model_name="microsoft/deberta-v3-base", num_labels=2):
        super(DeBERTaClassifier, self).__init__()
        self.deberta = AutoModel.from_pretrained(model_name)
        self.deberta.gradient_checkpointing_enable()
        self.classifier = nn.Linear(self.deberta.config.hidden_size, num_labels)
        self.mlm_head = nn.Linear(self.deberta.config.hidden_size, self.deberta.config.vocab_size)  # MLM head for pretraining
    
    def forward(self, input_ids, attention_mask, mlm=False):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        if mlm:
            return self.mlm_head(outputs.last_hidden_state)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        return logits

In [5]:
# Cell 5: Initialize Model, Optimizer, and Scheduler
model = DeBERTaClassifier().to(device)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# Learning rate scheduler
num_training_steps = 3 * len(welfake_df) // 32  # Adjusted for batch size 32
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

scaler = GradScaler()

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-5-17bdeab1d406>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [6]:
# Cell 6: Pre-training Loop
def pretrain_model(model, train_loader, epochs=1):
    model.train()
    mlm_criterion = nn.CrossEntropyLoss(ignore_index=-100)
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Pretraining Epoch {epoch+1}")
        for batch in progress_bar:
            inputs, labels = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast("cuda", dtype=torch.bfloat16):
                outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], mlm=True)
                loss = mlm_criterion(outputs.view(-1, model.deberta.config.vocab_size), labels.view(-1))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss / len(train_loader))

In [7]:
# Cell 7: Run Pre-training
pretrain_dataset = MLMDataset(pretrain_texts)
pretrain_loader = DataLoader(pretrain_dataset, batch_size=32, shuffle=True)
pretrain_model(model, pretrain_loader)

Pretraining Epoch 1: 100%|██████████| 2815/2815 [2:17:26<00:00,  2.93s/it, loss=2.26]  


In [ ]:
# Cell 8: Fine-tuning Loop
# Update Dataset with Reduced Sequence Length
class FakeNewsDataset(Dataset):
    def __init__(self, df, max_len=128):  # Reduced max_len from 256 to 128
        self.labels = torch.tensor(df["label"].values, dtype=torch.long)
        self.encodings = tokenizer(df["text"].astype(str).tolist(), 
                                   padding="max_length", truncation=True, 
                                   max_length=max_len, return_tensors="pt")
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items() if key != "token_type_ids"}, self.labels[idx]

# Reload Training Data with New Dataset Class
train_dataset = FakeNewsDataset(welfake_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Update Learning Rate Scheduler with Fewer Warmup Steps
num_training_steps = len(train_loader) * 1  # Only 1 epoch
num_warmup_steps = int(0.02 * num_training_steps)  # 2% Warmup

lr_scheduler = get_scheduler("linear", optimizer=optimizer, 
                             num_warmup_steps=num_warmup_steps, 
                             num_training_steps=num_training_steps)

# Updated Fine-Tuning Loop (1 Epoch)
def fine_tune_model(model, train_loader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    progress_bar = tqdm(train_loader, desc="Fine-tuning Epoch 1")
    
    for batch in progress_bar:
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)
        
        optimizer.zero_grad()
        with torch.amp.autocast("cuda"):
            outputs = model(**inputs)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        
        # Compute Accuracy
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / (total + 1e-8), accuracy=correct / total)

# Run Fine-Tuning (1 Epoch)
fine_tune_model(model, train_loader)

In [17]:
# Cell 9: Run Fine-tuning
train_dataset = FakeNewsDataset(welfake_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_model(model, train_loader)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-16-1c87c8f286f8>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Fine-tuning Epoch 1:   0%|          | 0/2255 [00:00<?, ?it/s]<ipython-input-16-1c87c8f286f8>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Fine-tuning Epoch 1:  17%|█▋        | 376/2255 [06:25<32:04,  1.02s/it, accuracy=0.928, loss=0.029] 


KeyboardInterrupt: 

In [ ]:
# Cell 10: Save and Evaluate Model
torch.save(model.state_dict(), "deberta_fakenews.pth")
print("Model saved successfully!")

In [ ]:
# Cell 11: Install Required Packages
!pip install rouge-score nltk scikit-learn

In [ ]:
# Cell 12: Model Evaluation and Testing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, log_loss, matthews_corrcoef
import numpy as np
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import torch

def evaluate_model(model, test_loader, task="classification"):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    total_loss = 0
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating Model"):
            inputs, labels = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)
            
            if task == "mlm":
                outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], mlm=True)
                loss = criterion(outputs.view(-1, model.deberta.config.vocab_size), labels.view(-1))
                total_loss += loss.item()
            else:
                logits = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
                probs = F.softmax(logits, dim=1)
                preds = torch.argmax(probs, dim=1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs[:, 1].cpu().numpy())

    if task == "mlm":
        perplexity = np.exp(total_loss / len(test_loader))
        print(f"MLM Perplexity: {perplexity:.4f}")
        return perplexity

    # Compute classification metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    roc_auc = roc_auc_score(all_labels, all_probs)
    logloss = log_loss(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)
    
    print("\nClassification Report:\n", classification_report(all_labels, all_preds))
    print("\nConfusion Matrix:\n", cm)
    print(f"\nMetrics: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1-Score={f1:.4f}, AUC-ROC={roc_auc:.4f}, Log Loss={logloss:.4f}, MCC={mcc:.4f}")

# Evaluate Fine-Tuned Model
evaluate_model(model, train_loader, task="classification")

# Evaluate Pretrained MLM Model
evaluate_model(model, pretrain_loader, task="mlm")

# Compute ROUGE and BLEU for MLM (Language Modeling)
rouge = Rouge()
bleu_scores = []
rouge_scores = {"rouge-1": [], "rouge-2": [], "rouge-l": []}

for i in range(10):  # Evaluate on 10 random sentences
    text = pretrain_texts[i]
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], mlm=True)
    
    predicted_tokens = torch.argmax(outputs, dim=-1)[0].cpu().numpy()
    predicted_text = tokenizer.decode(predicted_tokens, skip_special_tokens=True)
    
    reference = text.split()
    candidate = predicted_text.split()
    
    bleu = sentence_bleu([reference], candidate)
    bleu_scores.append(bleu)
    
    rouge_score = rouge.get_scores(predicted_text, text)[0]
    for key in rouge_scores.keys():
        rouge_scores[key].append(rouge_score[key]["f"])

# Output BLEU & ROUGE Scores
print(f"\nBLEU Score: {np.mean(bleu_scores):.4f}")
print(f"ROUGE-1: {np.mean(rouge_scores['rouge-1']):.4f}, ROUGE-2: {np.mean(rouge_scores['rouge-2']):.4f}, ROUGE-L: {np.mean(rouge_scores['rouge-l']):.4f}")
